<a href="https://colab.research.google.com/github/KudratBatta/MINI-PROJECTS/blob/main/email_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests-html lxml_html_clean

In [ ]:
from requests_html import AsyncHTMLSession
import re
import asyncio
import pandas as pd
from urllib.parse import urljoin, urlparse

#  Async scraping function
emails_set = set()
visited_urls = set()

async def scrape_emails_from_url(session, url, max_emails=20):
    global emails_set, visited_urls

    if len(emails_set) >= max_emails or url in visited_urls:
        return

    visited_urls.add(url)

    try:
        r = await session.get(url)
        await r.html.arender(timeout=20, sleep=1)

        text = r.html.text
        found_emails = set(re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', text))
        emails_set.update(found_emails)

        print(f"[+] {url} — {len(emails_set)} emails found")


        base_url = "{0.scheme}://{0.netloc}".format(urlparse(url))
        links = r.html.absolute_links
        internal_links = [link for link in links if link.startswith(base_url)]

        for link in internal_links:
            if len(emails_set) >= max_emails:
                break
            await scrape_emails_from_url(session, link, max_emails)

    except Exception as e:
        print(f"[!] Failed to fetch {url}: {e}")


async def main():
    session = AsyncHTMLSession()
    start_url = "https://www.thapar.edu"
    await scrape_emails_from_url(session, start_url, max_emails=20)


    emails_list = list(emails_set)[:20]
    df = pd.DataFrame(emails_list, columns=["Email"])
    df.to_csv("emails.csv", index=False)
    print("\n✅ Emails saved to 'emails.csv'")


await main()


[+] https://www.thapar.edu — 1 emails found
[+] https://www.thapar.edu/students/pages/scholarships — 1 emails found
[!] Failed to fetch https://www.thapar.edu/upload/files/Notice for Post Matric Scholarship (Govt.).pdf: net::ERR_ABORTED at https://www.thapar.edu/upload/files/Notice%20for%20Post%20Matric%20Scholarship%20(Govt.).pdf
[!] Failed to fetch https://www.thapar.edu/upload/files/dateextnsalumniandsch 2023-24.pdf: net::ERR_ABORTED at https://www.thapar.edu/upload/files/dateextnsalumniandsch%202023-24.pdf
[!] Failed to fetch https://www.thapar.edu/upload/files/Pro-Forma for Alumni MCM Scholarshipre(TIETDOAASCH(MCM ALUMNI)FT-002).pdf: net::ERR_ABORTED at https://www.thapar.edu/upload/files/Pro-Forma%20for%20Alumni%20MCM%20Scholarshipre(TIETDOAASCH(MCM%20ALUMNI)FT-002).pdf
[!] Failed to fetch https://www.thapar.edu/upload/files/Notice for Merit-cum-Means Scholarship for the year 2018-19.pdf: net::ERR_ABORTED at https://www.thapar.edu/upload/files/Notice%20for%20Merit-cum-Means%20Sch